In [5]:
# %pip install --upgrade --quiet pypdf
# %pip install --upgrade --quiet rapidocr-onnxruntime

In [12]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("C:/Users/kshit/Downloads/Bahdanau-Attention-Research-Paper.pdf", extract_images=True)
docs = loader.load()

In [36]:
type(docs)

list

In [37]:
print(docs[0])

page_content='Published as a conference paper at ICLR 2015
NEURAL MACHINE TRANSLATION
BY JOINTLY LEARNING TO ALIGN AND TRANSLATE
Dzmitry Bahdanau
Jacobs University Bremen, Germany
KyungHyun Cho Yoshua Bengio ∗
Universit´e de Montr´eal
ABSTRACT
Neural machine translation is a recently proposed approach to machine transla-
tion. Unlike the traditional statistical machine translation, the neural machine
translation aims at building a single neural network that can be jointly tuned to
maximize the translation performance. The models proposed recently for neu-
ral machine translation often belong to a family of encoder–decoders and encode
a source sentence into a ﬁxed-length vector from which a decoder generates a
translation. In this paper, we conjecture that the use of a ﬁxed-length vector is a
bottleneck in improving the performance of this basic encoder–decoder architec-
ture, and propose to extend this by allowing a model to automatically (soft-)search
for parts of a source sentence th

In [41]:
# Now here we want to use a text splitter to split the text into chunks

from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitters = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
        ".",
        " "
    ],
    chunk_size= 400,
    chunk_overlap = 20
)

In [42]:
# Text splitter receives argument as a list

doc = text_splitters.split_documents(docs) 

In [44]:
print(doc[0])

page_content='Published as a conference paper at ICLR 2015
NEURAL MACHINE TRANSLATION
BY JOINTLY LEARNING TO ALIGN AND TRANSLATE
Dzmitry Bahdanau
Jacobs University Bremen, Germany
KyungHyun Cho Yoshua Bengio ∗
Universit´e de Montr´eal
ABSTRACT
Neural machine translation is a recently proposed approach to machine transla-
tion. Unlike the traditional statistical machine translation, the neural machine' metadata={'source': 'C:/Users/kshit/Downloads/Bahdanau-Attention-Research-Paper.pdf', 'page': 0}


In [45]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings()
vector_store = FAISS.from_documents(doc, embeddings)

In [47]:
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from langchain_core.vectorstores import VectorStoreRetriever

retriever = VectorStoreRetriever(vectorstore= vector_store)
# 

groq_api_key = "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"

llm = ChatGroq(
    api_key= groq_api_key,
    model="mixtral-8x7b-32768",
    temperature= 0.8,
    max_retries= 2
)

In [48]:
retrievalQA = RetrievalQA.from_llm(llm= llm, retriever = retriever)
# we could have used retriever = "vector_store.as_retriever()" too

In [52]:
query = "What is the core information of the document? Provide answer upto 300 words."

retrievalQA({"query" : query}, return_only_outputs= True)

{'result': "The core information of the document revolves around the comparison of the translation capabilities of different systems, specifically Google Translate and two RNN (Recurrent Neural Network) models, RNNenc-50 and RNNsearch-50. The text highlights the superior performance of the RNNsearch-50 model in translating a long French sentence into English, as opposed to Google Translate, which made basic mistakes such as leaving out a closing quotation mark.\n\nThe French sentence speaks of Disney's initiatives to extend the lifespan of its series and build new relationships with digital audiences through increasingly important online platforms. The RNNsearch-50 model accurately translated the entire sentence, while Google Translate struggled with the task.\n\nFurthermore, the text touches upon the concept of machine translation and how it differs from traditional methods. It emphasizes the importance of online platforms for media companies like Disney in fostering connections with 